# 通过净利润的思想来挑选股票

In [1]:
import pandas as pd
import numpy as np
import pdb
import time

import datetime

import sys
sys.path.append("../utils/")
import token_util
pro = token_util.set_token()

*获取股票列表*

In [2]:
stock_list = pd.read_csv("../data_pulled/stock_date_delta_priceNone.csv")
print("stock_list.shape: ", stock_list.shape)
print("")
stock_list.head()

stock_list.shape:  (1946, 3)



,ts_code,name,pe_ttm
0,000005.SZ,世纪星源,68.4298
1,000007.SZ,全新好,123.8896
2,000008.SZ,神州高铁,20.2295
3,000009.SZ,中国宝安,58.8490
4,000012.SZ,南玻A,25.5406


### 成长股
三年内年净利润增长率最少大于10%，平均大于25% <br>
毛利率三年平均>40% <br>
净利率三年平均>20% <br>

*获取数据*

In [3]:
def get_data(ts_code, time_list):
    for i in range(len(time_list)):

        ret = pro.income(ts_code=ts_code, period='20'+str(time_list[i])+'1231', fields='ts_code,end_date,n_income,\
                         revenue,oper_cost')
        rets = pd.concat([rets, ret], ignore_index=True) if i != 0 else ret

    rets = rets.drop_duplicates()[-4:].reset_index(drop=True)
    rets['mao_li'] = (rets['revenue']-rets['oper_cost'])/rets['revenue']
    rets['jing_li'] = rets['n_income']/rets['revenue']
    
    return rets.drop(['oper_cost', 'revenue'], axis=1)

*进行筛选*

In [4]:
candidate_list = []
time_list = [15, 16, 17, 18, 19, 20]
times = 0
starttime = datetime.datetime.now()
len_stock = len(stock_list)

while True:
    
    # 发生中断后，防止遗漏
    start_index = 0
    
    for cur_index in range(start_index, len_stock):
        
        start_index = cur_index
        
        try:
            data = get_data(stock_list["ts_code"][cur_index], time_list)
        except:
            time.sleep(5)

        times += 1
        pass_data = True
        in_come_list = []

        maoli_mean = data['mao_li'][1:].mean()
        jingli_mean = data['jing_li'][1:].mean()

        if maoli_mean >= 0.4 and jingli_mean >= 0.2:

            for i in range(1, len(data)):
                n_income_last = data.iloc[i-1]["n_income"]
                n_income_cur = data.iloc[i]["n_income"]
                income_rate = (n_income_cur - n_income_last) / n_income_last
                in_come_list.append(income_rate)

                if n_income_cur > 0 and n_income_cur > n_income_last and income_rate >= 0.10:
                    continue
                else:
                    pass_data = False
                    break
        else:
            pass_data = False
        
        if len(in_come_list) == 3 and pass_data is True and np.mean(in_come_list) >= 0.25:
            candidate_list.append(stock_list["ts_code"][cur_index])
            
        if cur_index != 0 and cur_index % 100 == 0:
            endtime = datetime.datetime.now()
            print("task accomplish: {} %, cost time : {} minutes".format(round((cur_index+1)/len_stock, 2), \
                                            round((endtime - starttime).seconds / 60, 2)))
            
    if cur_index >= len_stock - 1:
        endtime = datetime.datetime.now()
        print("finish task !! total time : {} minutes".format(round((endtime - starttime).seconds / 60, 2)))
        break

task accomplish: 0.05 %, cost time : 4.22 minutes
task accomplish: 0.1 %, cost time : 8.72 minutes
task accomplish: 0.15 %, cost time : 13.38 minutes
task accomplish: 0.21 %, cost time : 17.62 minutes
task accomplish: 0.26 %, cost time : 22.18 minutes
task accomplish: 0.31 %, cost time : 26.77 minutes
task accomplish: 0.36 %, cost time : 31.43 minutes
task accomplish: 0.41 %, cost time : 35.97 minutes
task accomplish: 0.46 %, cost time : 40.15 minutes
task accomplish: 0.51 %, cost time : 44.8 minutes
task accomplish: 0.57 %, cost time : 49.28 minutes
task accomplish: 0.62 %, cost time : 53.5 minutes
task accomplish: 0.67 %, cost time : 58.22 minutes
task accomplish: 0.72 %, cost time : 62.62 minutes
task accomplish: 0.77 %, cost time : 66.82 minutes
task accomplish: 0.82 %, cost time : 71.57 minutes
task accomplish: 0.87 %, cost time : 75.92 minutes
task accomplish: 0.93 %, cost time : 80.17 minutes
task accomplish: 0.98 %, cost time : 84.93 minutes
finish task !! total time : 86.95 mi

In [5]:
len(candidate_list)

31

In [6]:
candidate_list

['000568.SZ',
 '002034.SZ',
 '002099.SZ',
 '002127.SZ',
 '002439.SZ',
 '002677.SZ',
 '002821.SZ',
 '002822.SZ',
 '002823.SZ',
 '002825.SZ',
 '002826.SZ',
 '002827.SZ',
 '002828.SZ',
 '002830.SZ',
 '002831.SZ',
 '002832.SZ',
 '002833.SZ',
 '002901.SZ',
 '600276.SH',
 '600436.SH',
 '600519.SH',
 '600779.SH',
 '603338.SH',
 '603387.SH',
 '603393.SH',
 '603429.SH',
 '603583.SH',
 '603589.SH',
 '603638.SH',
 '603658.SH',
 '603688.SH']

In [7]:
stock_list[(stock_list["ts_code"] == stock_list["ts_code"][cur_index])]

,ts_code,name,pe_ttm
1945,603999.SH,读者传媒,48.0831


In [21]:
times

1946